一些有用的链接

[slides](http://web.stanford.edu/class/cs20si/lectures/slides_01.pdf)

[note](http://web.stanford.edu/class/cs20si/lectures/notes_01.pdf)

[详细notebook](https://github.com/lightaime/cs20si/blob/master/1_overview_of_tensorflow.ipynb)

## Overview of Tensorflow


In [1]:
import tensorflow as tf
import numpy as np

## Graphs and Session
tf中将操作分为了定义和执行量部分，分别对应的就是 Graphs 和 Session

我们先定义计算的模型即图，然后在session中执行得到结果

### Tensor
Tensor即张量，就是数学上的张量的概念，我们平时一般接触到的就是二维、三维的向量，更高维度就是张量了，当然张量是更高层的概念，因此也包含了0，1，2，3维

In [2]:
a = tf.add(3,5)
print(a)
print(a.op)

Tensor("Add:0", shape=(), dtype=int32)
name: "Add"
op: "Add"
input: "Add/x"
input: "Add/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



我们可以看到tf.add操作不是我们平时直观的算术操作，他返回的是一个Tensor，然后其捕获的是`Add`操作，并且有两个输入

要想执行操作，我们需要在Session中执行

In [3]:
with tf.Session() as sess:
    print(sess.run(a))

8


!note：我们通过像文件操作符一样的 with 关键字，可以省去 sess.close() 操作

下面再介绍一个操作

![](./images/l1-mul.png)

In [4]:
x = 2
y = 3
op1 = tf.add(x, y)
op2 = tf.multiply(x,y)
op3 = tf.pow(op2, op1)
print(op3)
assert op3.graph is tf.get_default_graph()

Tensor("Pow:0", shape=(), dtype=int32)


当我们在tf中不指定graph的时候，默认都是用的 `tf.get_default_graph()`

In [5]:
graph = tf.get_default_graph()
print(graph.get_operations())
print(graph.version)

[<tf.Operation 'Add/x' type=Const>, <tf.Operation 'Add/y' type=Const>, <tf.Operation 'Add' type=Add>, <tf.Operation 'Add_1/x' type=Const>, <tf.Operation 'Add_1/y' type=Const>, <tf.Operation 'Add_1' type=Add>, <tf.Operation 'Mul/x' type=Const>, <tf.Operation 'Mul/y' type=Const>, <tf.Operation 'Mul' type=Mul>, <tf.Operation 'Pow' type=Pow>]
10


在tf中，graph包含了 Operation 和 Tensor

> A Graph contains a set of tf.Operation objects, which represent units of computation; 
> and tf.Tensor objects, which represent the units of data that flow between operations.

那我们怎么覆盖默认的graph呢？

In [6]:
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(2)
    y = tf.constant(3)
    op1 = tf.add(x, y)
    op2 = tf.multiply(x, y)
    useless = tf.multiply(x, op1) # this multipy operation wolud not be run in case 1
    op3 = tf.pow(op2, op1)
    assert op3.graph is graph
print(graph.get_operations())
print(graph.version)

[<tf.Operation 'Const' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'Add' type=Add>, <tf.Operation 'Mul' type=Mul>, <tf.Operation 'Mul_1' type=Mul>, <tf.Operation 'Pow' type=Pow>]
6


In [7]:
# case 1
with tf.Session(graph = graph) as sess:
    op3 = sess.run(op3)
    print(op3)
    print(useless) # 我们会发现我 useless 没有被执行，因此tf只有用到的才会去执行

7776
Tensor("Mul_1:0", shape=(), dtype=int32)


In [8]:
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(2)
    y = tf.constant(3)
    op1 = tf.add(x, y)
    op2 = tf.multiply(x, y)
    useless = tf.multiply(x, op1) # run this operation
    op3 = tf.pow(op2, op1)
print(graph.get_operations())
print(graph.version)

[<tf.Operation 'Const' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'Add' type=Add>, <tf.Operation 'Mul' type=Mul>, <tf.Operation 'Mul_1' type=Mul>, <tf.Operation 'Pow' type=Pow>]
6


In [9]:
# case 2
with tf.Session(graph = graph) as sess:
    op3, not_useless = sess.run([op3, useless])
    print(op3)
    print(not_useless)

7776
10


## 定义计算图

In [10]:
graph = tf.Graph()
with graph.as_default():
    variable = tf.Variable(42, name='foo')
    initialize = tf.global_variables_initializer()
    assign = variable.assign(13)
print(graph.get_operations())
print(graph.version)

[<tf.Operation 'foo/initial_value' type=Const>, <tf.Operation 'foo' type=VariableV2>, <tf.Operation 'foo/Assign' type=Assign>, <tf.Operation 'foo/read' type=Identity>, <tf.Operation 'init' type=NoOp>, <tf.Operation 'Assign/value' type=Const>, <tf.Operation 'Assign' type=Assign>]
7


上面我们计算了一个变量，定义变量是因为我们在后续会对变量就行更新，譬如后续的 variable.assign(13) 操作

In [15]:
print(graph)
print(variable)
print(initialize)
print(assign)

Tensor("foo/read:0", shape=(), dtype=int32)
name: "init"
op: "NoOp"
input: "^foo/Assign"

Tensor("Assign:0", shape=(), dtype=int32_ref)


我们可以看到此处 variable是一个 read tensor，会需要我们将初始值赋给它，这就需要用到 tf.global_variables_initializer()

> Returns an Op that initializes global variables.

我们也需要将其在session中执行

In [16]:
with tf.Session(graph = graph) as sess:
    sess.run(initialize)
    sess.run(assign)
    print(sess.run(variable))

13


In [19]:
# 如果我们不进行初始化操作呢？
with tf.Session(graph = graph) as sess:
    print(sess.run(variable))
    # FailedPreconditionError: Attempting to use uninitialized value foo

In [20]:
# 只执行 initialize, variable 则返回42，多余的操作不会执行
with tf.Session(graph = graph) as sess:
    sess.run(initialize)
    print(sess.run(variable))

42
